In [1]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

In [2]:
RHO_DEFAULT = 0.01
M_DEFAULT = 1

#Problem one
def target(x, y):
    return np.array([np.sin(x) + y])

def constraint(x, y):
    return np.sin(x)*np.sin(y) + 0.95

#Problem 2
def target_2(x, y):
    return np.array([x + y])

def constraint_2_1(x, y):
    return -0.5 * np.sin(2*np.pi * (x**2 - 2*y)) - x - 2*y + 1.5

def constraint_2_2(x, y):
    return x**2 + y**2 - 1.5


#YOUR OWN PROBLEM HERE
def target_3(x, y):
    raise NotImplemented
def constraint_3(x,y):
    return NotImplemented

def regulariser(x: np.array, z: np.array, y : np.array, rho = 0.01, M = 1):
    
    return rho / (2*M) * (np.linalg.norm(x - z + y/rho) ** 2)


In [19]:
from scipy.stats import norm
import warnings

    
def ADMMBO(pbounds : dict, target : callable, constraints : np.array, regulariser : callable, 
           num_constraints : int, num_inits_f : int, num_inits_constraint : np.array,
           init_ys : np.array = None, init_zs : np.array = None, init_xs : np.array = None,
           rho : float = 1e-1, M : int = 20, epsilon : float = 0.05, max_iter_outer = 3,
           max_iter_OPT : int = 5, max_iter_FEAS : int = 5):
    
    """
        params
        z: np.array for constraint
        x: np.array for value of x from k+1 iteration
        y: np.array for lambda of lagrangian for kth iteration
        rho: convergence parameter rho
        M : some sufficiently large number (hyperparameter) 
    """
    #Renaming for brevity
    n = num_inits_f
    m = num_inits_constraint
    
    assert num_constraints == m.shape[0]
    
    """
                                                        INITIALISATION
    ------------------------------------------------------------------------------------------------------------------
    
    """
    dim_space = len(pbounds)
    
    utility = UtilityFunction(kind="ei", kappa=2.5, xi=1e-6)
    
    
    #There will be one BayesianOptimisation function for each constraint, and these are saved in an array
    c_optims = []
    
    for i in range(num_constraints):
        new_c_optim = BayesianOptimization(
                #Note the lack of the regulariser here, and therefore will require using a non-standard
            #expectation-improvement procedure
                    f=constraints[i],
                    pbounds=pbounds,
                    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
                )
        c_optims.append(new_c_optim)
    
    
    #Initialise evaluation arrays (in paper F = {x, f(x)}, here F = {f(x)})
        #Same goes for C : in paper, C = {z, c(z)}, here C = {c(z)}
    F = np.zeros(n)
    Cs = np.zeros((num_constraints, np.max(m)))
    
    #This is if xs were initialised outside the function called (not recommended currently!)
    if not init_xs:
        init_xs = np.zeros((n, dim_space))
        #Generate the initial x points and evaluate all these points
        for i in range(n):
            #Because no points have been registered, this generates random points within bounds
            x_i = c_optims[0].suggest(utility)
            init_xs[i] = list(x_i.values())
            F[i] = target(**x_i)
    else:
        assert init_xs.shape[0] == num_inits_f
        assert init_xs.shape[1] == dim_space
        
        for i in range(n):
            F[i] = target(init_xs[i])
    
    if not init_zs:
        init_zs = np.zeros((num_constraints, np.max(m), dim_space))
        # Generate but do not evaluate all the constraint points zs (as these are evaluated later)
        for j in range(num_constraints):
            for k in range(m[j]):
                c_optim = c_optims[j]
                z_i = c_optim.suggest(utility)
                init_zs[j,k] = list(z_i.values())
            
            #Now register the points (this is done behind the scenes with probe. 
            #Lazy = True just saves some computation by only calculating inverses when necessary)
            for k in range(m[j]):
                point = init_zs[j,k]
                Cs[j,k] = constraints[j](*point)
                c_optim.register(
                        params = point,
                        target = Cs[j,k]
                        )
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                c_optim._gp.fit(c_optim._space.params, c_optim._space.target)
    else:
        assert init_zs.shape[0] == num_constraints
        assert init_zs.shape[1] == np.max(m)
        assert init_zs.shape[2] == dim_space
        assert False, "not implemented yet"
    if not init_ys:
        init_ys = np.random.uniform(0, 1, size = (num_constraints, dim_space))
    else:
        assert init_ys.shape[0] == num_constraints, "number of initialised lambda values must be equal to number of constraints"
    
    """
                                                        MAIN LOOP
    ------------------------------------------------------------------------------------------------------------------
    
    """
    
    solved = False
    k = 0
    xs = init_xs
    zs = init_zs
    ys = init_ys
    
    #Use first slice of zs as current best value of zs
    best_z = zs[:, 0, :]
    
    while k < max_iter_outer and not solved:
        best_x, xs, F = run_opt(target, regulariser, xs, F, pbounds, best_z, ys, rho, max_iter = max_iter_OPT)
        
        best_z, zs, Cs, ys, c_optims, m, rho, solved = run_feas(
            constraints=constraints, regulariser=regulariser, 
            z_mins_prev=best_z, m=m, x=best_x, 
            Cs=Cs, c_optims=c_optims, zs=zs, ys=ys, 
            pbounds=pbounds, rho=rho, M=M, epsilon=epsilon,
            max_iter=max_iter_FEAS)
        k+=1
    
    if solved:
        return best_x
    else:
        print('Did not stop at criterion, finding approximate answer ... ')
        return find_approx(xs=xs, F=F, zs=zs, Cs=Cs, c_optims=c_optims, pbounds = pbounds)
            

def run_opt(target : callable, regulariser : callable, xs : np.array, F : np.array, 
            pbounds : dict, zs : np.array, ys : np.array, rho : float, max_iter = 1):
    """
                                                       OPT INNER LOOPS
    ------------------------------------------------------------------------------------------------------------------
    
    """
    assert len(ys) == len(zs)
    
    #We need a new temp_optim object for each run because the posterior will be different based on each new z
    temp_optim = BayesianOptimization(
            f=None,
            pbounds=pbounds
        )

    U = np.zeros(len(xs))

    #This updates the GP posterior
    for i in range(xs.shape[0]):
        q_sum = np.sum(np.array([regulariser(xs[i], zs[j], ys[j], rho = rho) for j in range(zs.shape[0])]))
        U[i] = F[i] + q_sum

        #This here registers the points without fitting the GP
        try:
            temp_optim.register(
                params = xs[i],
                target = -U[i]
            )
        except:
            pass
            #This is fine, and should only be uncommented if interested in knowing 
            #print(f"TARGET: we already have {xs[i]}, but it tried adding")
    
    #Fit all the points added
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        temp_optim._gp.fit(temp_optim._space.params, temp_optim._space.target)
    
    utility = UtilityFunction(kind="ei", kappa=2.5, xi=1e-6) 
    for _ in range(max_iter):
        next_point = temp_optim.suggest(utility)

        xs = np.concatenate((xs, np.array(list(next_point.values())).reshape(1, xs.shape[1])), axis = 0)
        
        F = np.concatenate((F, target(**next_point)))
        
        q_sum = np.sum(np.array([regulariser(xs[-1], zs[j], ys[j], rho = rho) for j in range(zs.shape[0])]))
        U[i] = F[-1] + q_sum
        try:
            #Will add -U[-1], else it will try and maximise, but we want to minimise
            temp_optim.register(
                params = next_point,
                target = -U[-1]
            )
        except:
            pass
            #This is fine, and should only be uncommented if interested in knowing 
            #print(f"TARGET: we already have {xs[i]}, but it tried adding")
    
    #Find the best value after all inner loops are completed
    argmin = np.argmin(U)
    xmin = xs[argmin]
    
    #This isn't needed anymore, so save some memory
    del temp_optim
    
    return (xmin, xs, F)
    
def run_feas(constraints : np.array, regulariser : callable, z_mins_prev : np.array, m : np.array,
            x : np.array, Cs : np.array, c_optims : np.array, zs : np.array, ys : np.array, 
            pbounds : dict, rho : float, M : int, epsilon : float, max_iter = 1):
    """
                                                      FEAS INNER LOOPS
    ------------------------------------------------------------------------------------------------------------------
    
    """

    #Very unintuitive naming for rs and ss but it basically means the "r's" and "s's" 
    # used in the paper for later determining if we're done or not
    z_mins = np.zeros((zs.shape[0], zs.shape[2])); rs = np.zeros((zs.shape[0], zs.shape[2])); 
    ss = np.zeros((zs.shape[0], zs.shape[2]));

    #For each constraint
    for j in range(zs.shape[0]):
        c_optim = c_optims[j]
        #Evaluates H over all known points
        H = (np.int64(Cs[j][0:m[j]] > 0)
                + np.array([regulariser(x, zs[j, a ,:], ys[j], rho, M) for a in range(m[j])]))
        h_plus = np.min(H)
        
        utility = UtilityFunction(kind="constraint", kappa=None, xi=0, 
                                  x_constraint = x, y_constraint = ys[j], rho = rho, M = M, h_plus = h_plus)
        
        for iteration in range(max_iter):
            
            #Add something to h
            #If we're at the max number of constraint m, we need to concat, otherwise just assign
            next_point = c_optim.suggest(utility)
            
            if m[j] == np.max(m):
                zs = np.concatenate((zs, np.zeros((zs.shape[0], 1, zs.shape[2]))), axis = 1)
                Cs = np.concatenate((Cs, np.zeros((Cs.shape[0], 1))), axis = 1)
            
            m[j]+=1
            zs[j, m[j]-1] = np.array(list(next_point.values()))
            c_eval = constraints[j](*list(next_point.values()))
            Cs[j, m[j]-1] = c_eval         

            H_new = int(c_eval > 0) + regulariser(x, zs[j, m[j]-1, :], ys[j], rho, M)
            
            if m[j]-1 == H.shape[0]:
                H = np.concatenate((H, np.array([H_new])))
            else:
                H[m[j]-1] = H_new
            
            try:
                c_optim.register(
                    params = next_point,
                    target = c_eval
                )
            except:
                pass
                #This is fine, and should only be uncommented if interested in knowing 
            #print(f"CONSTRAINT: tried adding {next_point} but looks like we already have that point")        
        
        #Update the parameters for the next outer loop iteration
        z_mins[j, :] = zs[j, np.argmin(H), :]
        ys[j, :] += rho * (x - z_mins[j, :])
        rs[j, :] = x - z_mins[j, :] 
        ss[j, :] = -rho * (z_mins[j, :] - z_mins_prev[j, :])
    
    is_solved = (np.linalg.norm(rs) < epsilon) & (np.linalg.norm(ss) < epsilon)
    
    
    #Changing rho as recommended by the paper where we have hardcoded mu and tao here
    mu = 10
    tao = 2
    
    if np.linalg.norm(rs) > mu * np.linalg.norm(ss):
        rho*=tao
    elif np.linalg.norm(rs) < mu * np.linalg.norm(ss):
        rho/=tao
    else:
        pass
    return z_mins, zs, Cs, ys, c_optims, m, rho, is_solved

def find_approx(xs : np.array, F : np.array, zs : np.array, Cs: np.array, 
                c_optims : np.array, pbounds : dict, delta : float = 0.1):
    """
                                          FIND APPROXIMATE ANSWER IF STOP CRITERION NOT MET
    ------------------------------------------------------------------------------------------------------------------
    
    """    
    #Just using this object to get the GP mean later
    final_optim = BayesianOptimization(
            f=None,
            pbounds=pbounds
        )
        
    #Check through current xs to find candidate
    
    #Initialises
    F_min = np.inf
    approx_x_min = xs[0]
    
    #Sets the GP posterior
    for i in range(xs.shape[0]):
        try:
            final_optim.register(
                params = xs[i],
                target = F[i]
            )
        except:
            pass
        
        #Check if it's a candidate for a minimum
        if F[i] < F_min:
            
            #Check if high prob of satisifying constraint
            satisfied = 1
            for c_optim in c_optims:
                mean, std = c_optim._gp.predict(xs[i].reshape(1,-1), return_std=True)
                satisfied *= int(norm.cdf(-mean/std) > 1 - delta)
            
            #Only if all constraints are likely satisfied then update it as the minimum candidate
            if satisfied == 1:
                F_min = F[i]
                approx_x_min = xs[i]
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        final_optim._gp.fit(final_optim._space.params, final_optim._space.target)
    
    #Check through zs
    feasibles = zs[np.where(Cs < 0)]
    for point in feasibles:
        #If the expected value of the function at this point is better than our current optimum, update
        E_f = final_optim._gp.predict(point.reshape(1,-1))
        satisfied = 1
        for c_optim in c_optims:
            mean, std = c_optim._gp.predict(xs[i].reshape(1,-1), return_std=True)
            satisfied *= int(norm.cdf(-mean/std) > 1 - delta)
        
        if E_f < F_min and satisfied == 1:
            F_min = E_f
            approx_x_min = point

    if F_min == np.inf:
        print("no feasible point was found in the search, try increasing no. of iterations")
        return None
    del final_optim
    #Return the best estimate
    return approx_x_min

## Run Subproblem 1:
I.e target = $\min_{x,y \in B = [0,6]^2} \sin(x) + y$

subject to the constraint $\sin(x)*\sin(y) + 0.95 \leq 0$

In [12]:
constraints = np.array([constraint])
num_constraints = 1
num_inits_f = 20
num_inits_constraint = np.array([20])
pbounds = {'x': (0, 6), 'y': (0, 6)}

answer = ADMMBO(pbounds, target, constraints, regulariser, num_constraints, num_inits_f, 
                 num_inits_constraint, rho = 0.1, max_iter_outer=80, max_iter_OPT=5, max_iter_FEAS=5)
x_min = answer
print(x_min)

[4.71960024 1.30745788]


Check if constraints were indeed satisfied and value of the target funciton at this point

In [14]:
paper_answer_1 = np.array([4.7, 1.3])
print("The value of the constraint at the found solution was " + str(constraint(*x_min)) + 
        " and was therefore " + ("not " if constraint(*x_min) > 0 else "") + "satisfied")

print("The value of the target function was " + str(target(*x_min)[0]) + " which is " + 
      ("lower " if target(*x_min) < target(*paper_answer_1) else "higher ") + "than that found by the paper, with a solution of " + str(target(*paper_answer_1)[0]))

The value of the constraint at the found solution was -0.015501239409989154 and was therefore satisfied
The value of the target function was 0.3074838786799571 which is higher than that found by the paper, with a solution of 0.3000767424358992


## Run Subproblem 2:
I.e target = $\min_{x,y \in B = [0,1]^2} x + y$

subject to the constraints:

$-0.5 * \sin(2 \pi(x^2 - 2y)) - x - 2y + 1.5 \leq 0$

$ x^2 + y^2 - 1.5 \leq 0$

We run our iterations a fewer number of times than the paper did, just for demonstration purposes. Feel free to increase the number of iterations.

One may need to restart the kernel as the underlying library stores a lot in cache and needs to be reset sometimes

In [22]:
constraints = np.array([constraint_2_1, constraint_2_2])
num_constraints = 2
num_inits_f = 10
num_inits_constraint = np.array([10,10])
pbounds = {'x': (0, 1), 'y': (0, 1)}
answer = ADMMBO(pbounds, target_2, constraints, regulariser, num_constraints, num_inits_f, 
                 num_inits_constraint, rho = 0.1, max_iter_outer=30, max_iter_OPT=5, max_iter_FEAS=5)
x_min = answer
print(x_min)

Did not stop at criterion, finding approximate answer ... 
[0.20245288 0.43318168]


In [23]:
paper_answer_2 = np.array([0.19, 0.415])
print("The value of the first constraint at the found solution was " + str(constraint_2_1(*x_min)) + 
        " and was therefore " + ("not " if constraint_2_1(*x_min) > 0 else "") + "satisfied")

print("The value of the second constraint at the found solution was " + str(constraint_2_2(*x_min)) + 
        " and was therefore " + ("not " if constraint_2_2(*x_min) > 0 else "") + "satisfied")

print("The value of the target function was " + str(target(*x_min)[0]) + " whereas the paper found " 
      + str(paper_answer_2) + " which gives a value of " + str(target(*paper_answer_2)[0]))

The value of the first constraint at the found solution was -0.013781722832890031 and was therefore satisfied
The value of the second constraint at the found solution was -1.2713664624918577 and was therefore satisfied
The value of the target function was 0.6342544001462149 whereas the paper found [0.19  0.415] which gives a value of 0.6038588949765006


In [9]:
x_min

array([0.31141383, 0.47308905])

0.04113417016370513